In [1]:
from fm_flux import FM_CALC
import numpy as np
import matplotlib.pyplot as plt

In [2]:
catalog_path = './full_earth_FM_moonrak.npy'

In [3]:
obsID = np.loadtxt('/home/himanshu/pawsey/phase2/moon/scripts/obsIDs_Oct_7_10_2023', dtype=np.int32)

In [4]:
FM_process = FM_CALC(catalog_path=catalog_path)

In [5]:
altaz_dist = FM_process.get_altaz_n_dist(obsID = obsID[0], FM_loc=None, save_as_array=False,)

In [6]:
altaz_dist.shape

(171225, 3)

In [7]:
station_cont=FM_process.get_station_RFIcontributions(obsID= obsID[0], altaz_array=altaz_dist,\
                                                 randomise_zero_power_stations=True,\
                                       save_as_array=False, )

In [9]:
station_cont.shape

(20477, 8)

In [10]:
FMflux = FM_process.get_FM_RFI_flux(obsID=obsID, stations_array=station_cont, save_as_array=False,)

IndexError: index 20481 is out of bounds for axis 0 with size 20477